# Write C++ wrapper for ghostbasil

`ml cmake/3.11.1 gcc/12.1.0 julia/1.8.4`

1. We will be using [CxxWrap.jl](https://github.com/JuliaInterop/CxxWrap.jl) to call C++ code within Julia. 
2. First, we must create a `libcxxwrap_julia_jll` JLL library, following [its github instructions](https://github.com/JuliaInterop/libcxxwrap-julia). 
3. Once it's done, I got a folder `/home/groups/sabatti/.julia/dev/libcxxwrap_julia_jll/override`

## Hello world example

Here is my `hello.cpp` file:
```cpp
#include <string>
#include "jlcxx/jlcxx.hpp"

std::string greet()
{
   return "hello, world";
}

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
  mod.method("greet", &greet);
}
```

And here is the `CMakeLists.txt` file:

```cmake
project(HelloWorld)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(JlCxx)
get_target_property(JlCxx_location JlCxx::cxxwrap_julia LOCATION)
get_filename_component(JlCxx_location ${JlCxx_location} DIRECTORY)
set(CMAKE_INSTALL_RPATH "${CMAKE_INSTALL_PREFIX}/lib;${JlCxx_location}")

message(STATUS "Found JlCxx at ${JlCxx_location}")

add_library(hello SHARED hello.cpp)

target_link_libraries(hello JlCxx::cxxwrap_julia)

install(TARGETS
  hello
LIBRARY DESTINATION lib
ARCHIVE DESTINATION lib
RUNTIME DESTINATION lib)
```

Given these 2 files, we build the shared library (`.io`) file as
```
mkdir build && cd build
cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_PREFIX_PATH=/home/groups/sabatti/.julia/dev/libcxxwrap_julia_jll/override /home/users/bbchu/ghostbasilwrap
cmake --build . --config Release
```
Finally, we can call hello world in Julia:

In [1]:
# Load the module and generate the functions
module CppHello
using CxxWrap
@wrapmodule("/home/users/bbchu/ghostbasilwrap/build/lib/libhello")

function __init__()
    @initcxx
end
end

# Call greet and show the result
CppHello.greet()

"hello, world"

## Pass vector from Julia into C++, calculate the sum, and return answer

First create `sumarray.cpp`

```cpp
#include "jlcxx/jlcxx.hpp"
#include <vector>
#include <iostream>
using namespace std;

double cpp_mat_sum(jlcxx::ArrayRef<double, 2> x) {
    cout << &x[0] << endl; // print memory address of x
    double total = 0;
    for(auto xij : x) {
        total += xij;
    }
    return total;
}

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
    mod.method("cpp_mat_sum", &cpp_mat_sum);
}
```

Then create `CMakeLists.txt` file

```cmake
project(Examples)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(JlCxx)
get_target_property(JlCxx_location JlCxx::cxxwrap_julia LOCATION)
get_filename_component(JlCxx_location ${JlCxx_location} DIRECTORY)
set(CMAKE_INSTALL_RPATH "${CMAKE_INSTALL_PREFIX}/lib;${JlCxx_location}")

message(STATUS "Found JlCxx at ${JlCxx_location}")

add_library(hello SHARED hello.cpp)
add_library(sumarray SHARED sumarray.cpp)

target_link_libraries(hello JlCxx::cxxwrap_julia)
target_link_libraries(sumarray JlCxx::cxxwrap_julia)

install(TARGETS
  hello sumarray 
LIBRARY DESTINATION lib
ARCHIVE DESTINATION lib
RUNTIME DESTINATION lib)
```
Given these 2 files, we build the shared library (`.io`) file as
```
mkdir build && cd build
cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_PREFIX_PATH=/home/groups/sabatti/.julia/dev/libcxxwrap_julia_jll/override /home/users/bbchu/ghostbasilwrap
cmake --build . --config Release
```
Finally, we can call both `hello` and `sumarray`:

In [1]:
module CppHello
using CxxWrap
@wrapmodule("/home/users/bbchu/ghostbasilwrap/build/lib/libhello")

function __init__()
    @initcxx
end
end

# Call greet and show the result
CppHello.greet()

"hello, world"

In [2]:
module CppSumArray
using CxxWrap
@wrapmodule("/home/users/bbchu/ghostbasilwrap/build/lib/libsumarray")

function __init__()
    @initcxx
end
end

x = randn(10000, 10000)
CppSumArray.cpp_mat_sum(x)

0x7f76084ff040


-2957.256808475831

In [3]:
sum(x)

-2957.2568084772065

In [4]:
pointer(x)

Ptr{Float64} @0x00007f76084ff040